In [1]:
'''
Equal-Weight S&P 500 Index Fund

Introduction & Library Imports
The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.
'''
import numpy as np
import pandas as pd
import math
import yfinance as yf
import time

'''
Importing Our List of Stocks
The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope.
'''
stocks = pd.read_csv("sp_500_stocks_cleaned.csv")
print(stocks.head())

'''
Adding Our Stocks Data to a Pandas DataFrame
The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. 
Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.
'''
my_columns = ['Ticker', 'Stock Price', 'Market Captilization', 'Number Of Shares to Buy', 'Amount Invested']
final_dataframe = pd.DataFrame(columns=my_columns)

# Fix tickers for Yahoo Finance format (BRK.B -> BRK-B)
def fix_ticker(symbol):
    if "." in symbol:
        return symbol.replace(".", "-")
    return symbol

# --- NEW: Use batch download for speed ---
tickers_list = [fix_ticker(sym) for sym in stocks['Ticker']]
data = yf.download(tickers_list, period="1d", group_by='ticker', threads=True, progress=False)

# --- Loop through tickers to fetch prices & market cap ---
final_df = []
failed_symbols = []

for ticker_symbol in tickers_list:
    try:
        # Fetch price
        if ticker_symbol in data:
            price = data[ticker_symbol]['Close'].iloc[-1]
        else:
            price = "N/A"

        # Fetch market cap
        ticker = yf.Ticker(ticker_symbol)
        market_cap = ticker.fast_info.get("market_cap")
        if market_cap is None:
            try:
                info = ticker.info
                market_cap = info.get("marketCap", "N/A")
            except:
                market_cap = "N/A"

        if price == "N/A":
            failed_symbols.append(ticker_symbol)

        final_df.append([ticker_symbol, price, market_cap, 0, 0.0])

        # Small delay to avoid API rate limits
        time.sleep(0.05)

    except:
        final_df.append([ticker_symbol, "N/A", "N/A", 0, 0.0])
        failed_symbols.append(ticker_symbol)

# Convert to DataFrame
final_dataframe = pd.DataFrame(final_df, columns=my_columns)

# Remove failed symbols
final_dataframe = final_dataframe[final_dataframe['Stock Price'] != "N/A"].reset_index(drop=True)

'''
Portfolio Allocation
'''
portfolio_size = input("Enter Value of Portfolio: ")
try:
    portfolio_value = float(portfolio_size)
except ValueError:
    print("That's not a number! Using default portfolio = $100000")
    portfolio_value = 100000

position_size = portfolio_value / len(final_dataframe)

for i in range(len(final_dataframe)):
    price = final_dataframe.loc[i, 'Stock Price']
    if price != "N/A":
        shares_to_buy = math.floor(position_size / price)
        final_dataframe.loc[i, 'Number Of Shares to Buy'] = shares_to_buy
        final_dataframe.loc[i, 'Amount Invested'] = round(shares_to_buy * price, 2)

'''
Save to Excel
'''
final_dataframe.to_excel("equal_weight_sp500.xlsx", index=False)
print("✅ Portfolio saved to 'equal_weight_sp500.xlsx'")

'''
Preview the first 10 stocks
'''
print(final_dataframe.head(10))


  Ticker
0      A
1    AAL
2    AAP
3   AAPL
4   ABBV


C:\Users\AUSER\AppData\Local\Temp\ipykernel_276\3464546061.py:45: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers_list, period="1d", group_by='ticker', threads=True, progress=False)


Enter Value of Portfolio:  100000


✅ Portfolio saved to 'equal_weight_sp500.xlsx'
  Ticker  Stock Price Market Captilization  Number Of Shares to Buy  \
0      A   153.600006          43632345088                        1   
1    AAL    13.100000           8647133184                       16   
2    AAP    50.930000           3056932864                        4   
3   AAPL   275.920013        4094760583168                        0   
4   ABBV   229.509995         405632417792                        0   
5    ABT   127.190002         221368991744                        1   
6    ACN   243.619995         151739367424                        0   
7   ADBE   318.730011         135205273600                        0   
8    ADI   239.399994         117774123008                        0   
9    ADM    58.650002          28185415680                        3   

   Amount Invested  
0           153.60  
1           209.60  
2           203.72  
3             0.00  
4             0.00  
5           127.19  
6             0.00  
7  

In [ ]:
pip install yfinance
